In [ ]:
#import os ; os.environ['KERAS_BACKEND']='theano'
%run init.ipy
#import logging    
#logging.getLogger('keras').setLevel(logging.INFO)

from utils import *
import miregularizer2

# opts = dict(
#     sfx='v8train',
#     mi_grad_trainable = True,
#     nbepoch = 50,
#     batch_size = 128,
#     patiencelevel = 50,
#     do_pretrain=True,
#     N = None,
#     EntropyEstimateN = 2000,
#     HIDDEN_DIM = 20,
# )

# opts = dict(
#     sfx='v10train',
#     mi_grad_trainable = True,
#     nbepoch = 50,
#     batch_size = 128,
#     patiencelevel = 50,
#     N = None,
#     EntropyEstimateN = 2000,
#     HIDDEN_DIM = 20,
# )
import keras.optimizers
mnist_mlp_base = dict( # gets 1.28-1.29 training error
    do_MI = False,
    do_validate_on_test = True,
    mi_grad_trainable   = True,
    nbepoch             = 30,
    batch_size          = 128,
    EntropyEstimateN    = 2000,
    #HIDDEN_DIMS = [800,800],
    #hidden_acts = ['relu','relu'],
    HIDDEN_DIMS    = [800,800,256],
    hidden_acts    = ['relu','relu','linear'],
)

# Achieves 1.37-1.4, but note that this with noise in the predictions also
# opts = mnist_mlp_base.copy()
# opts['sfx'] = 'mnist_v1'
# opts['do_MI'] = True  

opts = mnist_mlp_base.copy()
opts['sfx'] = 'mnist_v1'
opts['do_MI'] = True  
opts['test_phase_noise'] = False

if 'do_pretrain' not in opts:
    opts['do_pretrain'] = False


def lrscheduler(epoch):
    if epoch < 10:
        return 0.001
    elif epoch < 20:
        return 0.0005
    else:
        return 0.0001

# from eve import *
# optimizer=Eve(),    
# if isinstance(opts['optimizer'], Eve):
#     print "EveMonitor callback"
#     callbacks.append(EveMonitor())


In [ ]:
#N=10000
d=load_mnist(max_train_items=opts.get('N', None), max_test_items=opts.get('N', None)) # , keep_classes = [0,5])

In [ ]:
from keras.layers.normalization import BatchNormalization

model = Sequential()

for hndx, hdim in enumerate(opts['HIDDEN_DIMS']):
    if 'hidden_inits' in opts:
        cinit = opts['hidden_inits'][hndx]
    else:
        cinit = 'he_uniform'
    model.add(Dense(hdim, 
                    input_dim=d.train.X.shape[1] if hndx == 0 else None, 
                    activation=opts['hidden_acts'][hndx],
                    init=cinit))
    
kdelayer, noiselayer, micomputer = None, None, None
    
callbacks = [keras.callbacks.LearningRateScheduler(lrscheduler),]

if opts.get('do_MI', True):
    kdelayer   = miregularizer2.KDEParamLayer(init_logvar=-5)
    model.add(kdelayer)
    regularize_mi_input = d.train.X
    if opts['EntropyEstimateN'] is not None:
        regularize_mi_input = randsample(regularize_mi_input, opts['EntropyEstimateN'])

    def get_noise_input_func(pd):
        import tensorflow as tf
        d = tf.constant(pd)
        for layerndx, layer in enumerate(model.layers):
            d = layer(d)
        return d
        
    noiselayer = miregularizer2.GaussianNoise2(init_logvar=-10, kdelayer=kdelayer, 
                                               regularize_mi_input=regularize_mi_input, 
                                               init_alpha=0., 
                                               get_noise_input_func=get_noise_input_func,
                                               trainable=opts['mi_grad_trainable'],
                                               test_phase_noise=opts.get('test_phase_noise', True))
    model.add(noiselayer)

    callbacks.append(miregularizer2.KDETrain(entropy_train_data=regularize_mi_input, kdelayer=kdelayer))
    if not opts['mi_grad_trainable']:
        callbacks.append(miregularizer2.NoiseTrain(traindata=d.train, noiselayer=noiselayer))
    callbacks.append(miregularizer2.ReportVars(noiselayer=noiselayer, kdelayer=kdelayer))

    
#model.add(Dense(HIDDEN_DIM, activation='tanh')) # , activity_regularizer=MIRegularizer(0, gaussian_var)))
model.add(Dense(d.train.nb_classes, init='glorot_uniform', activation='softmax')) # , activity_regularizer=MIRegularizer(0, gaussian_var)))

def compilefunc():
    optimizer = opts.get('optimizer','adam')
    print "Using optimizer", optimizer
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    if noiselayer is not None and hasattr(noiselayer, 'logvar'):
        K.set_value(noiselayer.logvar, -10)
    if kdelayer is not None and hasattr(kdelayer, 'logvar'):
        K.set_value(kdelayer.logvar, -10)
    
if opts.get('do_validate_on_test', False):
    validation_split = None
    validation_data = (d.test.X, d.test.Y)
    early_stopping = None
else:
    validation_split = 0.2
    validation_data = None
    from keras.callbacks import EarlyStopping
    callbacks.append( EarlyStopping(monitor='val_loss', patience=opts['patiencelevel']) )
    

compilefunc()
fit_args = dict(
    x=d.train.X,
    y=d.train.Y,
    verbose=2,
    batch_size=opts['batch_size'],
    callbacks=callbacks,
    nb_epoch=opts['nbepoch'],
    validation_split = validation_split,
    validation_data = validation_data,
)


In [ ]:
#%%timeit -n 1 -r 1
if opts.get('do_pretrain', True):
    if noiselayer is not None:
        K.set_value(noiselayer.mi_regularizer.alpha, 0.0)

    inithist = model.fit(**fit_args)

In [ ]:
if True:
    fname = "models/initmodel-%s.h5"%opts['sfx']
    print "Saving %s"% fname
    model.save_weights(fname)
    fname = "models/opts-%s.txt"%opts['sfx']
    print "Saving %s"% fname
    with open(fname, 'w') as f:
        f.write(str(opts))


In [ ]:
#alphavals = np.linspace(0, 5, 20, endpoint=True)
#alphavals = np.exp(np.linspace(0, np.log(6), 20, endpoint=True))-1
#alphavals = np.exp(np.linspace(0, np.log(3.), 30, endpoint=True))-1
#alphavals = np.exp(np.linspace(np.log(5), np.log(10.), 10, endpoint=True))-1
#alphavals = np.exp(np.linspace(0, np.log(10.), 10, endpoint=True))-1
#alphavals = np.exp(np.linspace(0, np.log(opts['max_alpha']+1), 10, endpoint=False))-1
alphavals = np.linspace(0, opts.get('max_alpha', 1.0), 20, endpoint=False)

# no alpha 
# 15s - loss: 2.7612e-04 - acc: 1.0000 - val_loss: 0.0799 - val_acc: 0.9872

alphavals = [1e-3,]

#del saved_hist
#saved_hist = {}
if 'saved_hist' not in locals():
    print "Initializing saved_hist"
    saved_hist = {}
#saved_hist = {}
for alpha in alphavals:
    print "**************** Doing alpha=%.8f, %s ****************" % (alpha, opts['sfx'])
    if alpha in saved_hist:
        continue

    compilefunc()
    model.load_weights("models/initmodel-%s.h5"%opts['sfx'])
        
    if noiselayer is not None :
        K.set_value(noiselayer.mi_regularizer.alpha, alpha)

    hist = model.fit(**fit_args)
    saved_hist[alpha] = {'history':hist.history, 
                         'endlogs': miregularizer2.get_logs(model, d, kdelayer, noiselayer, max_entropy_calc_N=opts['EntropyEstimateN'])}
    fname = "models/fitmodel-%s-%0.5f.h5"%(opts['sfx'],alpha)
    print "saving to %s"%fname
    model.save_weights(fname)
    
    if True:
        savedhistfname='models/savedhist-%s.dat'%opts['sfx']
        with open(savedhistfname, 'wb') as f:
            cPickle.dump(saved_hist, f)
        print 'updated', savedhistfname

    print 
    print
raise Exception('Done')

In [ ]:
"""
compilefunc()
#model.load_weights("models/fitmodel-%s.h5"%sfx)
K.set_value(noiselayer.mi_regularizer.alpha, 400.)

hist = model.fit(d.train.X, d.train.Y, nb_epoch=200,
                 batch_size=10., verbose=2, # validation_split=0.1, 
                 callbacks=[kdetraincb, reportcb,]  # noisetraincb, 
                )
print miregularizer2.get_logs(model, d, kdelayer, noiselayer)
"""

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
#fitmodel1.00000.h5
activity_trn = {}
activity_tst = {}
for alphaval in [0,0.5,1.2]:
    fname = "models/fitmodel-v10train-%0.5f.h5"%alphaval
    compilefunc()
    model.load_weights(fname)
    print fname
    
    #activity = get_activations(model, -2, d.X_train)[0]
    #get_activations = K.function([model.layers[0].input, K.learning_phase()], [noiselayer.output,])
    #activations = get_activations([X_batch,0])
    #activity = get_activations([d.X_train, 1])
    #plt.figure()
    #plot_activity(activity, colors=d.y_train, size=1) # , doPCA=False)
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [noiselayer.input,])
    
    activity_trn[alphaval]=get_activations([d.train.X,])[0]
    activity_tst[alphaval]=get_activations([d.test.X,])[0]
    print 'COV determinant:', np.linalg.det(np.cov(activity_trn[alphaval].T))
    print 'COV diagonals:', np.diag(np.cov(activity_trn[alphaval].T))
    print activity_trn[alphaval].shape

In [ ]:
fig = plt.figure(figsize=(15,14))
for ndx, alphaval in enumerate(sorted(activity_trn.keys())): # enumerate(["fitmodel.h5", "fitmodel1.00000.h5","fitmodel2.00000.h5"]): #"fitmodel1.50000.h5", 
    for trntstndx in range(2):
        ax = fig.add_subplot(len(activity_trn),2,2*ndx+trntstndx+1) # , projection='3d')
        print "Doing", ndx, trntstndx
        #plt.subplot(1,4,ndx+1)
        if trntstndx == 0:
            activitydata, titlestr, labels = activity_trn, 'trn', d.train
        else:
            activitydata, titlestr, labels = activity_tst, 'tst', d.test
        #if activitydata[alphaval].shape[1] == 2:
        #    plt.scatter(activitydata[alphaval][:,0], activitydata[alphaval][:,1], c=labels.y, edgecolors='none')
        
        plotcolors = labels.y
        plotdata = activitydata[alphaval]
        if False:
            plotdata, ixs = randsample(plotdata, 2000, return_ixs=True)
            plotcolors = plotcolors[ixs]
        
        plot_activity(plotdata, method='ica', colors=plotcolors, size=2, dims=2)# , opts=dict(whiten=False))
        plt.title(r'$\alpha = %0.3f$ (%s)' % (alphaval,  titlestr))
        plt.xticks([])
        plt.yticks([])
    
#plt.savefig('imgs/hidden.pdf',bbox_inches='tight')

In [ ]:
import sklearn

In [ ]:
sklearn.__version__
